In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 加载文档内容（每行为一个 chunk）
with open("data/test.txt", "r", encoding="utf-8") as f:
    docs = [line.strip() for line in f if line.strip()]

# 加载 embedding 模型（可替换为 bge/text2vec）
model = SentenceTransformer("all-mpnet-base-v2")

print(docs)

# 生成向量（默认维度 384）
embeddings = model.encode(docs, normalize_embeddings=True)
print(f"共生成 {len(embeddings)} 个向量，每个维度为 {embeddings.shape[1]}")

/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['第七条：公司是企业法人，有独立的法人财产。', '第八条：公司以其全部财产对公司债务承担责任。', '董事会对公司的经营决策具有最终权力。', '股东大会是公司的最高权力机构。', '李彦宏称大模型成本每年降低90%。', '乌军大批直升机击落多架俄无人机。', '王力宏回应是否想找新伴侣。', '饺子不知道观众怎么想出的藕饼cp。', '加沙停火协议关键时刻生变。', 'The weather is lovely today。', "It's so sunny outside。", 'He drove to the stadium。']
共生成 12 个向量，每个维度为 768


In [2]:
import chromadb  # ChromaDB 向量数据库

# 初始化 ChromaDB 客户端 (持久化到本地目录)
chroma_client = chromadb.PersistentClient(path="chroma_db")

# 创建或获取集合 (相当于数据库表)
collection = chroma_client.get_or_create_collection(
    name="news_articles",  # 集合名称
    metadata={"hnsw:space": "cosine"}  # 使用余弦相似度
)

In [5]:
# 准备文档 ID (需要唯一标识符)
document_ids = [str(i) for i in range(len(docs))]  # 生成 ["0", "1", ..., "4"]

# 将文档插入数据库
collection.add(
    ids=document_ids,               # 唯一ID列表
    embeddings=embeddings, # 文本向量列表
    documents=docs             # 原始文本列表
)

# 查询文本
query_text = "明星"
# 生成查询向量
query_embedding = model.encode([query_text], normalize_embeddings=True)

# 执行相似性查询
results = collection.query(
    query_embeddings=query_embedding,  # 查询向量
    n_results=10                         # 返回前2个最相似结果
)

# 打印查询结果
print("查询内容:", query_text)
print("最相似结果:")
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"相似度 {score:.4f}: {doc}")

查询内容: 明星
最相似结果:
相似度 0.5348: 加沙停火协议关键时刻生变。
相似度 0.6042: 王力宏回应是否想找新伴侣。
相似度 0.6571: 股东大会是公司的最高权力机构。
相似度 0.6646: 第八条：公司以其全部财产对公司债务承担责任。
相似度 0.6667: 乌军大批直升机击落多架俄无人机。
相似度 0.6767: 董事会对公司的经营决策具有最终权力。
相似度 0.6813: 饺子不知道观众怎么想出的藕饼cp。
相似度 0.6826: 李彦宏称大模型成本每年降低90%。
相似度 0.6850: 第七条：公司是企业法人，有独立的法人财产。
相似度 0.7148: It's so sunny outside。
